In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, ResNet50, VGG16
from tensorflow.keras.layers import BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import itertools

In [ ]:
# Constants
SEED = 1888
IMG_SIZE = [176, 208]
BATCH_SIZE = 32
CLASS_LIST = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']
DIR_INPUT = './data-initial/Dataset/'
DIR_WORK = './'
DIR_MODELS = os.path.join(DIR_WORK, 'models')
DIR_TRAIN = os.path.join(DIR_WORK, 'resampled', 'train')
DIR_VAL = os.path.join(DIR_WORK, 'resampled', 'val')
DIR_TEST = os.path.join(DIR_WORK, 'resampled', 'test')

tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
# Data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    DIR_TRAIN,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED
)

val_gen = val_datagen.flow_from_directory(
    DIR_VAL,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED
)

test_gen = val_datagen.flow_from_directory(
    DIR_TEST,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED,
    shuffle=False
)

In [ ]:
# Define metrics
metrics = [tf.keras.metrics.AUC(name=f'auc_{class_name}') for class_name in CLASS_LIST]

# Define the build_transfer_model function
def build_transfer_model(conv_base, dropout, dense_nodes, learn_rate, metrics):
    model = tf.keras.Sequential()
    model.add(conv_base)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(BatchNormalization())
    model.add(tf.keras.layers.Dense(dense_nodes, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(len(metrics), activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                  loss='categorical_crossentropy', 
                  metrics=metrics)
    return model

In [ ]:
# Fine-tune the pretrained model
def fine_tune_model(model, trainable_layers):
    for layer in model.layers[-trainable_layers:]:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = True

# Function to plot training metrics
def plot_training_metrics(history, model, test_gen, y_actual, y_pred, class_list):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

    print(classification_report(y_actual, y_pred, target_names=class_list))
    cm = confusion_matrix(y_actual, y_pred)
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_list))
    plt.xticks(tick_marks, class_list, rotation=45)
    plt.yticks(tick_marks, class_list)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [ ]:
# Custom Model 2: GoogleNet
# Get base
conv_base_google = InceptionV3(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))

# Fine-tune the top layers of the InceptionV3 base
fine_tune_model(conv_base_google, trainable_layers=100)

# Build and compile the model
model_google = build_transfer_model(conv_base_google, dropout=0.5, dense_nodes=624, learn_rate=0.00001, metrics=metrics)

# Train
EPOCHS_GOOGLE = 100 
tic_google = time.perf_counter()
history_google = model_google.fit(
    train_gen,
    epochs=EPOCHS_GOOGLE,
    validation_data=val_gen,
    verbose=2
)

# Get class predictions
y_prob_google = model_google.predict(test_gen)
y_pred_google = y_prob_google.argmax(axis=-1)

# Get actual classes
y_actual_google = test_gen.classes

# Plot training metrics for GoogleNet model
plot_training_metrics(history_google, model_google, test_gen, y_actual_google, y_pred_google, CLASS_LIST)

# Compute OvA AUC for each class for GoogleNet
y_one_hot_google = label_binarize(y_actual_google, classes=np.arange(len(CLASS_LIST)))
y_prob_google = model_google.predict(test_gen)
for i, class_name in enumerate(CLASS_LIST):
    auc_value = roc_auc_score(y_one_hot_google[:, i], y_prob_google[:, i])
    print(f'OvA AUC for {class_name}: {auc_value}')

# Save Model
model_google.save(os.path.join(DIR_MODELS, 'model_google'))

# Time
toc_google = time.perf_counter()
print("Total Time for GoogleNet: {} mins".format(round((toc_google-tic_google)/60, 2)))

# Custom Model 3: Vision Transformer
# Use a suitable implementation for Vision Transformer
class VisionTransformer(tf.keras.Model):
    def __init__(self):
        super(VisionTransformer, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(len(CLASS_LIST), activation='softmax')

    def call(self, inputs):
        x = self.flatten(inputs)
        return self.dense(x)

conv_base_vit = VisionTransformer()

# Build and compile the model
model_vit = build_transfer_model(conv_base_vit, dropout=0.5, dense_nodes=624, learn_rate=0.00001, metrics=metrics)

# Train
EPOCHS_VIT = 100 
tic_vit = time.perf_counter()
history_vit = model_vit.fit(
    train_gen,
    epochs=EPOCHS_VIT,
    validation_data=val_gen,
    verbose=2
)


In [ ]:
# Get class predictions
y_prob_vit = model_vit.predict(test_gen)
y_pred_vit = y_prob_vit.argmax(axis=-1)

# Get actual classes
y_actual_vit = test_gen.classes

# Plot training metrics for Vision Transformer model
plot_training_metrics(history_vit, model_vit, test_gen, y_actual_vit, y_pred_vit, CLASS_LIST)

# Compute OvA AUC for each class for Vision Transformer
y_one_hot_vit = label_binarize(y_actual_vit, classes=np.arange(len(CLASS_LIST)))
y_prob_vit = model_vit.predict(test_gen)
for i, class_name in enumerate(CLASS_LIST):
    auc_value = roc_auc_score(y_one_hot_vit[:, i], y_prob_vit[:, i])
    print(f'OvA AUC for {class_name}: {auc_value}')

# Save Model
model_vit.save(os.path.join(DIR_MODELS, 'model_vit'))

# Time
toc_vit = time.perf_counter()
print("Total Time for Vision Transformer: {} mins".format(round((toc_vit-tic_vit)/60, 2)))

# Custom Model 4: ResNet34
# Get base
conv_base_resnet34 = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))

# Fine-tune the top layers of the ResNet34 base
fine_tune_model(conv_base_resnet34, trainable_layers=100)

# Build and compile the model
model_resnet34 = build_transfer_model(conv_base_resnet34, dropout=0.5, dense_nodes=624, learn_rate=0.00001, metrics=metrics)

# Train
EPOCHS_RESNET34 = 100 
tic_resnet34 = time.perf_counter()
history_resnet34 = model_resnet34.fit(
    train_gen,
    epochs=EPOCHS_RESNET34,
    validation_data=val_gen,
    verbose=2
)


In [ ]:
# Get class predictions
y_prob_resnet34 = model_resnet34.predict(test_gen)
y_pred_resnet34 = y_prob_resnet34.argmax(axis=-1)

# Get actual classes
y_actual_resnet34 = test_gen.classes

# Plot training metrics for ResNet34 model
plot_training_metrics(history_resnet34, model_resnet34, test_gen, y_actual_resnet34, y_pred_resnet34, CLASS_LIST)

# Compute OvA AUC for each class for ResNet34
y_one_hot_resnet34 = label_binarize(y_actual_resnet34, classes=np.arange(len(CLASS_LIST)))
y_prob_resnet34 = model_resnet34.predict(test_gen)
for i, class_name in enumerate(CLASS_LIST):
    auc_value = roc_auc_score(y_one_hot_resnet34[:, i], y_prob_resnet34[:, i])
    print(f'OvA AUC for {class_name}: {auc_value}')

# Save Model
model_resnet34.save(os.path.join(DIR_MODELS, 'model_resnet34'))

# Time
toc_resnet34 = time.perf_counter()
print("Total Time for ResNet34: {} mins".format(round((toc_resnet34-tic_resnet34)/60, 2)))

In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import itertools

In [ ]:
# Constants
SEED = 1888
IMG_SIZE = [176, 208]
BATCH_SIZE = 32
CLASS_LIST = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']
DIR_INPUT = './data-initial/Dataset/'
DIR_WORK = './'
DIR_MODELS = os.path.join(DIR_WORK, 'models')
DIR_TRAIN = os.path.join(DIR_WORK, 'resampled', 'train')
DIR_VAL = os.path.join(DIR_WORK, 'resampled', 'val')
DIR_TEST = os.path.join(DIR_WORK, 'resampled', 'test')

tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
# Data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    DIR_TRAIN,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED
)

val_gen = val_datagen.flow_from_directory(
    DIR_VAL,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED
)

test_gen = val_datagen.flow_from_directory(
    DIR_TEST,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED,
    shuffle=False
)

In [ ]:
# Define metrics
metrics = [tf.keras.metrics.AUC(name=f'auc_{class_name}') for class_name in CLASS_LIST]

# Define the build_transfer_model function
def build_transfer_model(conv_base, dropout, dense_nodes, learn_rate, metrics):
    model = tf.keras.Sequential()
    model.add(conv_base)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(BatchNormalization())
    model.add(tf.keras.layers.Dense(dense_nodes, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(len(metrics), activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                  loss='categorical_crossentropy', 
                  metrics=metrics)
    return model

In [ ]:
# Fine-tune the pretrained model
def fine_tune_model(model, trainable_layers):
    for layer in model.layers[-trainable_layers:]:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = True

# First Custom Model: Base + Additional Capacity (93 epochs)
# Get base
conv_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))

# Fine-tune the top layers of the InceptionV3 base
fine_tune_model(conv_base, trainable_layers=100)

# Build and compile the model
model_93 = build_transfer_model(conv_base, dropout=0.5, dense_nodes=624, learn_rate=0.00001, metrics=metrics)

# Train
EPOCHS_93 = 110 
tic = time.perf_counter()
history_93 = model_93.fit(
    train_gen,
    epochs=EPOCHS_93,
    validation_data=val_gen,
    verbose=2
)

In [ ]:
# Get class predictions
y_prob_93 = model_93.predict(test_gen)
y_pred_93 = y_prob_93.argmax(axis=-1)

# Get actual classes
y_actual_93 = test_gen.classes

# Function to plot training metrics
def plot_training_metrics(train_hist, model, test_gen, y_actual, y_pred, classes):
    test_results = model.evaluate(test_gen, verbose=False)
    test_loss = test_results[0]
    test_metric = test_results[1]  # Adjust this if the relevant metric is at a different index
    results = round(test_metric, 2) * 100
    results_title = "\n Model AUC on Test Data: {}%".format(results)
    print(results_title)
    print(len(results_title) * "-")
    print(classification_report(y_actual, y_pred, target_names=classes))
    history_dict = train_hist.history
    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']
    
    # Combine AUC values for all classes
    auc_values = [history_dict[f'auc_{class_name}'] for class_name in classes]
    val_auc_values = [history_dict[f'val_auc_{class_name}'] for class_name in classes]
    
    # Calculate mean AUC across all classes
    mean_auc_values = np.mean(auc_values, axis=0)
    mean_val_auc_values = np.mean(val_auc_values, axis=0)
    
    epochs = range(1, len(history_dict[f'auc_{classes[0]}']) + 1)
    max_auc = np.max(mean_val_auc_values)
    min_loss = np.min(val_loss_values)

    plt.subplots(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.plot(epochs, loss_values, 'bo', label='Training loss')
    plt.plot(epochs, val_loss_values, 'cornflowerblue', label='Validation loss')
    plt.title('Validation Loss by Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.axhline(y=min_loss, color='darkslategray', linestyle='--')
    plt.legend()
    plt.subplot(1, 3, 2)
    plt.plot(epochs, mean_auc_values, 'bo', label='Training AUC')
    plt.plot(epochs, mean_val_auc_values, 'cornflowerblue', label='Validation AUC')
    plt.plot(epochs, [results / 100] * len(epochs), 'darkmagenta', linestyle='--', label='Test AUC')
    plt.title('Validation AUC by Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.axhline(y=max_auc, color='darkslategray', linestyle='--')
    plt.legend()

    cm = confusion_matrix(y_actual, y_pred)
    plt.subplot(1, 3, 3)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.BuPu)
    plt.title("Confusion Matrix \n AUC:{}%".format(results))
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    threshold = cm.max() / 2.
    for r, c in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(c, r, format(cm[r, c], 'd'), horizontalalignment="center", color="white" if cm[r, c] > threshold else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

# Plot training metrics
plot_training_metrics(history_93, model_93, test_gen, y_actual_93, y_pred_93, CLASS_LIST)

# Compute OvA AUC for each class
class_names = CLASS_LIST

y_one_hot_93 = label_binarize(y_actual_93, classes=np.arange(len(class_names)))
y_prob_93 = model_93.predict(test_gen)
for i, class_name in enumerate(class_names):
    auc_value = roc_auc_score(y_one_hot_93[:, i], y_prob_93[:, i])
    print(f'OvA AUC for {class_name}: {auc_value}')

# Save Model
model_93.save(os.path.join(DIR_MODELS, 'model_custom_93_epochs'))

# Time
toc = time.perf_counter()
print("Total Time for 93 Epochs: {} mins".format(round((toc-tic)/60, 2)))


In [ ]:
import os
import os.path
from os import path
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from IPython.display import display, clear_output

In [ ]:
# Load the trained model
model_path = os.path.join('models', 'model_added_capacity')
# Update with the actual path to your model
model = load_model(model_path)

# Function to preprocess the uploaded image and make a prediction
def predict_image():
    file_path = filedialog.askopenfilename(
        title="Select MRI Image",
    )

    if file_path:
        img = image.load_img(file_path, target_size=(176, 208))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Normalize pixel values to the range [0, 1]

        result = model.predict(img_array)
        result_class = np.argmax(result)
        class_names = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very Mild Demented']
        result_text.set(f"Prediction: {class_names[result_class]}")

        # Display the uploaded image
        img = Image.open(file_path)
        img = img.resize((200, 150))
        img = ImageTk.PhotoImage(img)
        image_label.config(image=img)
        image_label.image = img

In [1]:
# Create the main GUI window
root = tk.Tk()
root.title("Alzheimer's Detection")

# Create GUI components
upload_button = tk.Button(root, text="Upload MRI Image", command=predict_image, bg = "red")
result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, font=("Helvetica", 14))
image_label = tk.Label(root)

# Pack GUI components
upload_button.pack(pady=10)
result_label.pack(pady=10)
image_label.pack(pady=10)

# Run the GUI application
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/kashishkapoor/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/w5/_vl7tvj16f7c3190jn_03clc0000gn/T/ipykernel_87618/786032371.py", line 26, in predict_image
    img = image.load_img(file_path, target_size=(176, 208))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kashishkapoor/anaconda3/lib/python3.11/site-packages/keras/src/utils/image_utils.py", line 423, in load_img
    img = pil_image.open(io.BytesIO(f.read()))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kashishkapoor/anaconda3/lib/python3.11/site-packages/PIL/Image.py", line 3283, in open
    raise UnidentifiedImageError(msg)
PIL.UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x2992da8e0>
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/kashishkapoor/anac

1/1 [==============================] - 1s 610ms/step
